In [2]:
import pandas as pd
import edhec_risk_kit_111 as erk

# Load the data
edhec = pd.read_csv(
    "data/edhec-hedgefundindices.csv",
    index_col=0,
    parse_dates=True,
    dayfirst=True
)
edhec = edhec / 100

# Filter for 2000 onwards
rets = edhec.loc[edhec.index >= "2000-01-01", "Distressed Securities"]

# Parametric Gaussian VaR at 1% (as a positive number)
var_1pct = erk.var_gaussian(rets, level=1, modified=False)
print(f"Monthly Parametric Gaussian VaR at 1% (Distressed Securities): {abs(var_1pct) * 100:.2f}")

Monthly Parametric Gaussian VaR at 1% (Distressed Securities): 3.14


In [3]:
# Cornish-Fisher VaR at 1% (as a positive number)
var_cf_1pct = erk.var_gaussian(rets, level=1, modified=True)
print(f"Cornish-Fisher 1% VaR (Distressed Securities): {abs(var_cf_1pct) * 100:.2f}")

Cornish-Fisher 1% VaR (Distressed Securities): 4.97


In [4]:
# Historic VaR at 1% (as a positive number)
hist_var_1pct = -rets.quantile(0.01)

# Convert to percent
print("Historic 1% VaR (Distressed Securities):", round(hist_var_1pct * 100, 2))

Historic 1% VaR (Distressed Securities): 4.26


In [5]:
industries = ["Books", "Steel", "Oil", "Mines"]
n = len(industries)
w_ew = 1 / n
print("Weight of Steel in EW Portfolio (%):", round(w_ew * 100, 1))

Weight of Steel in EW Portfolio (%): 25.0


In [6]:
import edhec_risk_kit_111 as erk

# Load industry returns
ind = erk.get_ind_returns()

# Filter for 2013-2017
ind_2013_2017 = ind["2013":"2017"]

# Select relevant industries
industries = ["Books", "Steel", "Oil", "Mines"]
rets = ind_2013_2017[industries]

# Estimate expected returns and covariance
er = erk.annualize_rets(rets, 12)
cov = rets.cov()

# Calculate MSR weights
rf = 0.10
w_msr = erk.msr(rf, er, cov)

# Find the largest weight
largest_weight = w_msr.max()
print("Largest MSR Portfolio Weight (%):", round(largest_weight * 100, 1))

Largest MSR Portfolio Weight (%): 100.0


In [7]:
# Find the industry with the largest weight
largest_weight_industry = industries[w_msr.argmax()]
print("Industry with largest MSR Portfolio Weight:", largest_weight_industry)

Industry with largest MSR Portfolio Weight: Steel


In [8]:
# Count non-zero weights in MSR portfolio
non_zero_count = (w_msr != 0).sum()
print("Number of non-zero weights in MSR portfolio:", non_zero_count)

Number of non-zero weights in MSR portfolio: 3


In [9]:
# Calculate GMV weights
w_gmv = erk.gmv(cov)

# Find the largest weight
largest_gmv_weight = w_gmv.max()
print("Largest GMV Portfolio Weight (%):", round(largest_gmv_weight * 100, 2))

Largest GMV Portfolio Weight (%): 47.7


In [10]:
# Find the industry with the largest GMV weight
largest_gmv_industry = industries[w_gmv.argmax()]
print("Industry with largest GMV Portfolio Weight:", largest_gmv_industry)

Industry with largest GMV Portfolio Weight: Books


In [11]:
# Count non-zero weights in GMV portfolio
non_zero_gmv = (w_gmv != 0).sum()
print("Number of non-zero weights in GMV portfolio:", non_zero_gmv)

Number of non-zero weights in GMV portfolio: 4


In [12]:
# Define the list of industries
l = ["Books", "Steel", "Oil", "Mines"]

# Get 2018 returns and covariance matrix
cov_2018 = ind.loc["2018", l].cov()

# Use MSR weights from previous calculation (w_msr)
# Portfolio volatility (annualized)
def portfolio_vol(weights, covmat):
    return (weights.T @ covmat @ weights) ** 0.5

vol_msr_2018 = portfolio_vol(w_msr, cov_2018) * (12 ** 0.5)
print("Annualized Volatility of MSR Portfolio (2018, %):", round(vol_msr_2018 * 100, 2))

Annualized Volatility of MSR Portfolio (2018, %): 21.98


In [13]:
# Portfolio volatility (annualized) for GMV portfolio
vol_gmv_2018 = portfolio_vol(w_gmv, cov_2018) * (12 ** 0.5)
print("Annualized Volatility of GMV Portfolio (2018, %):", round(vol_gmv_2018 * 100, 2))

Annualized Volatility of GMV Portfolio (2018, %): 18.97
